In [3]:
from pyspark.sql.types import *

In [4]:
#Defining schema

labels = [
              ('Year', StringType()),
              ('Month', StringType()),
              ('DayofMonth', StringType()),
              ('DayOfWeek', StringType()),
              ('DepTime', StringType()),
              ('CRSDepTime', StringType()),
              ('ArrTime', StringType()),
              ('CRSArrTime', StringType()),
              ('UniqueCarrier', StringType()),
              ('FlightNum', StringType()),
              ('TailNum', StringType()),
              ('ActualElapsedTime', StringType()),
              ('CRSElapsedTime', StringType()),
              ('AirTime', StringType()),
              ('ArrDelay', StringType()),
              ('DepDelay', StringType()),
              ('Origin', StringType()),
              ('Dest', StringType()),
              ('Distance', StringType()),
              ('TaxiIn', StringType()),
              ('TaxiOut', StringType(),
              ('Cancelled', StringType())),
              ('CancellationCode', StringType()),
              ('Diverted', StringType()),
              ('CarrierDelay', StringType()),
              ('WeatherDelay', StringType()),
              ('NASDelay', StringType()),
              ('SecurityDelay', StringType()),
              ('LateAircraftDelay', StringType())
        ]
schema = StructType([StructField(x[0], x[1], True) for x in labels])

In [4]:
#Reading Streams

df = spark \
    .readStream \
    .format("parquet") \
    .option("header", True) \
    .schema(schema) \
    .option("maxFilesPerTrigger",1) \
    .load("./spark-warehouse/bucket_data_month_year")


#Creating Temp View table of streamed data
df.createOrReplaceTempView("temp_df")

In [5]:
#Giving the SQL Query

sql_df = spark.sql("SELECT Month, Year, count(*) FROM temp_df GROUP BY Month, Year")

In [7]:
def foreach_batch_function(df, epoch_id):
    df.show(10, False)

In [ ]:
#Streaming output

sql_df.writeStream.outputMode("update").foreachBatch(foreach_batch_function).start()
spark.streams.awaitAnyTermination()